In [4]:

import sys 
sys.path.append('..')

import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
import seaborn as sns


In [7]:
gru_loss_name = 'gru_lead1_stat2_loss.csv'
lstm_loss_name = 'lstm_lead1_dyn1_loss.csv'
ar_loss_name = 'ar_lead48_dyn2_loss.csv'

forecast lead 1

In [8]:
columns_to_sum = ['test_loss', 'test_loss_true', 'test_mse', 'test_mae']
clusters = ['WEBSTER', 'HIGH', 'TED', 'HAMILTON', 'RINCONADA', 'CAMBRIDGE', 'MPL', 'BRYANT']

df_gru = pd.read_csv("../data/LossCSVs/" + gru_loss_name, index_col=0)
print(df_gru.shape)
df_lstm = pd.read_csv("../data/LossCSVs/" + lstm_loss_name, index_col=0)
print(df_lstm.shape)


(33, 172)
(27, 172)


In [17]:
df_gru['run_no'] = np.repeat(np.arange(1,5),8)
#df_lstm['run_no'] = np.repeat(np.arange(1,8),8)

# We first sum each run, and then average across runs
df_gru_agg = df_gru.groupby('run_no').mean()
#f_lstm_agg = df_lstm.groupby('run_no').sum()

/var/folders/zw/yys6vm7506q010fwbsz_3djr0000gn/T/ipykernel_2342/573480410.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gru['run_no'] = np.repeat(np.arange(1,5),8)


In [18]:
print(np.round(df_gru_agg[columns_to_sum].mean(),2))
print(np.round(df_gru_agg[columns_to_sum].std(),2))

test_loss         0.46
test_loss_true   -0.09
test_mse          1.41
test_mae          0.75
dtype: float64
test_loss         0.01
test_loss_true    0.02
test_mse          0.10
test_mae          0.02
dtype: float64


In [44]:
print(np.round(df_lstm_agg[columns_to_sum].mean(),2))
print(np.round(df_lstm_agg[columns_to_sum].std(),2))

test_loss         6.84
test_loss_true   -0.71
test_mse          8.87
test_mae          5.67
dtype: float64
test_loss         0.02
test_loss_true    0.04
test_mse          0.48
test_mae          0.10
dtype: float64


forecast lead 48

In [30]:
df_ar_48 = pd.read_csv("../data/LossCSVs/" + ar_loss_name, index_col=0)
print(df_ar_48.shape)
df_ar_48 = df_ar_48[columns_to_sum]
df_ar_48['run_no'] = np.repeat(np.arange(1,8),8)
df_ar_48.head(10)

(56, 53)


,test_loss,test_loss_true,test_mse,test_mae,run_no
Name,,,,,
sunny-shape-467,1.118072,0.013835,2.201864,1.091379,1
hearty-darkness-466,1.453250,-0.078261,3.423576,1.339139,1
warm-vortex-465,1.284693,0.177268,1.560872,0.922951,1
silvery-breeze-464,0.617329,0.732744,2.409877,1.150609,1
comfy-wildflower-463,0.996154,0.939769,0.880992,0.774735,1
generous-galaxy-462,1.877385,-0.210363,9.958217,1.978139,1
lively-monkey-461,0.909120,0.835895,0.791044,0.749700,1
volcanic-serenity-460,1.483260,-0.487778,3.905221,1.380569,1
helpful-wildflower-459,1.170462,0.104843,2.998426,1.283635,2


In [33]:
df_ar_48.groupby('run_no').sum().agg(['mean', 'std'])

,test_loss,test_loss_true,test_mse,test_mae
mean,9.548014,1.679980,19.777939,8.816428
std,0.186763,0.236703,2.594258,0.382551
